## Calculate_SOM-BI_ERA5_codebook.ipynb

Calculates the blocking index for SOM-BI using the blocking index trained on ERA5 JJA 1979-2019 for 5x4 SOM nodes.

In [1]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import csv
import cartopy.crs as ccrs
import matplotlib.style as style
from scipy import stats
import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
 
def calc_blocked_days_clus(persis_thresh, SOM_nodes, node_cluster_set_test_str):
    """
    Calculate the number of blocked days in the cluster
    NB this function is fairly slow - perhaps a list comprehension could speed it up?
    SOM_nodes - list of labelled SOM nodes for each day [1,2,9...,7,6]
    blocked_days - arrangement of blocked days in the GTD
    persis_thresh - threshold for number of days to limit blocking
    node_cluster_set_test_str - the set of node clusters being tested, stored as a list of strings ['[1, 2, 5]', '[1, 4]', ... , '[1]']
    """
    #define the new set of blocked days
    blocked_days_clus = np.zeros((SOM_nodes.shape[0]))
    #set blocking for the five day clusters where the given SOM cluster is in the set of node clusters
    for i in range(SOM_nodes.shape[0]-persis_thresh):
        SOM_cluster = str(np.unique(SOM_nodes[i:i+persis_thresh]))
        if SOM_cluster in node_cluster_set_test_str:
            blocked_days_clus[i:i+persis_thresh] = 1
    return blocked_days_clus

def clus_to_str(clus):
    """
    Convert array of node numbers to a string
    """
    clus_str = str(clus.values)
    clus_str_corr = (clus_str.replace(".", "").replace("[", "").replace("]", "").replace(" ", "_").replace("__", "_"))
    if clus_str_corr[0] == "_":
        clus_str_corr = clus_str_corr[1:]
    return clus_str_corr
    
def Identify_BMU_from_codebook(codebook_da_reshaped, data_yr_reshaped, rownum, colnum):
    """
    When calculating the best matching unit from a given codebook (use the codebook from reanalysis)
    need to calculate the best matching units array
    output an array specifying the BMU with [rownum, colnum] for each day in the dataset
    
    codebook_da - codebook as a DataArray
    data_yr_reshaped - data from the xarray, reshaped so that the lat/lon values are along one axis
    """
    bmu_arr=[]
    for i, data_day in enumerate(data_yr_reshaped):
            #for each day define an arbitrarily large euclidean distance
            min_euclidean_distance = 1e12
            for rowcolnum, codebook in enumerate(codebook_da_reshaped):
                #calculate Euclidean distance between codebook and day in dataset
                euclidean_distance = (np.sum((np.array(data_day)-np.array(codebook))**2)**0.5)
                #find optimum minima by redefining the minimum euclidean distance
                if euclidean_distance < min_euclidean_distance:
                    #print(f"Redefining optimum for {rowcolnum}")
                    min_euclidean_distance = euclidean_distance
                    #print(f"min_euclidean_distance = {min_euclidean_distance} for {rowcolnum}")
                    min_rowcolnum = rowcolnum
            #print(f"min_rowcolnum = {min_rowcolnum}")
            if colnum > rownum:
                print(f"ERROR: colnum {colnum} > rownum {rownum}")
                return 1
            bmu_arr.append(min_rowcolnum+1)
            #bmu_arr.append([min_rowcolnum%colnum, min_rowcolnum//(rownum-(rownum-colnum))])
    return bmu_arr

def detrend(da, yrs_avg, moving_yrs):
    """
    Detrend the data using linear detrending over shorter time slices
    yrs_avg - the number of years to average over to calculate the detrended record (make an even number, and total years avteraged over will be yrs_avg + 1)
    moving_yrs - number of years to shift the moving average by
    """
    styr, endyr = int(da['time.year'].min()), int(da['time.year'].max())

    da_yr_detrend_arr = []

    for yr in np.arange(styr,endyr+1):
        if int(yrs_avg/2) != yrs_avg/2:
            print(f"yrs_avg = {yrs_avg}, needs to be even")
        else:
            if yr >= styr + yrs_avg/2 and yr < endyr - yrs_avg/2:
                #can have a window defined evenly both sides
                #print(f"mid {yr}")
                yrst_avg = yr - yrs_avg/2
                yrend_avg = yr + yrs_avg/2
            elif yr < styr + yrs_avg/2 and yr < endyr - yrs_avg/2:
                #near start of period
                #print(f"st {yr}")
                yrst_avg = styr
                yrend_avg = styr + yrs_avg
            elif yr >= styr + yrs_avg/2 and yr >= endyr - yrs_avg/2:
                #near end of period
                #print(f"end {yr}")
                yrst_avg = endyr - yrs_avg
                yrend_avg = endyr 
            else:
                print(f"yrs_avg period too long for detrending: {yrs_avg} too big for {styr}-{endyr} period")
            if yrend_avg-yrst_avg != yrs_avg:
                print(f"ERROR: {yrst_avg}-{yrend_avg}, {yrend_avg-yrst_avg} years not {yrs_avg} years")
            else:
                #run average
                da_sel = da.sel(time = np.isin(da['time.year'], np.arange(yrst_avg, yrend_avg+1)))
                y = fldmean_xr(da_sel.groupby("time.year").mean(axis=0))#fldmean_xr(da_sel)
                #print(da_sel_fldmean)
                #da_sel_fldmean = da_fldmean.sel(time = np.isin(da_fldmean['time.year'], np.arange(yrst_avg, yrend_avg+1)))
                #calculate time average and subtract using the trend derived across all days in the time period
                #y = da_sel_fldmean.groupby("time.year").mean()
                #print(y)
                x = np.arange(len(y))
                slope, intercept, r, p, se = stats.linregress(x, y)
                da_yr = da.sel(time = np.isin(da['time.year'], [yr]))
                da_yr_detrend = da_yr - intercept - slope*x[int(yr-yrst_avg)]
                da_yr_detrend_arr.append(da_yr_detrend)

    da_detrend = xr.concat(da_yr_detrend_arr, dim = "time")
    
    return da_detrend


def sel_JJA_extd(da):
    """
    Selects the JJA extd period +- 5 days.
    """
    da_MJJAS=da.sel(time = np.isin(da['time.month'], np.arange(5,10)))
    da_JJAextdS=da_MJJAS.sel(time=~(((da_MJJAS.time.dt.month == 5) & (da_MJJAS.time.dt.day < 28))))
    da_JJAextd=da_JJAextdS.sel(time=~(((da_JJAextdS.time.dt.month == 9) & (da_JJAextdS.time.dt.day > 4))))
    return da_JJAextd


def fldmean_xr(xr_da):
    """
    Calculate the area averaged mean from an xarray DataArray with coordinates time, latitude, longitude
    will need to be adapted if data is not 3d e.g. if there are other levels
    """
    if "latitude" in str(xr_da):
        lat_str, lon_str = "latitude", "longitude"
    else:
        lat_str, lon_str = "lat", "lon"
    #multiply the Dataarray by the relevant weights using cos(latitude)
    xr_da_latcorr = (xr_da*np.cos(np.radians(xr_da[lat_str])))    
    #calculate the weighted mean by summing across all the weighted values and dividing by the total sum of all weights appliced
    if xr.ufuncs.isnan(xr_da[0]).sum() > 0:
        print("Masked")
        #calculating the weighted mean for a masked array
        isnan = xr.ufuncs.isnan(xr_da[0])
        xr_da_latcorr = (xr_da*np.cos(np.radians(xr_da[lat_str])))
        weights_sum = ((isnan==0)*np.cos(np.radians(xr_da[lat_str]))).sum()
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1, skipna=True).sum(axis=1, skipna=True)/weights_sum
    
    if len(xr_da.shape) == 3:
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1).sum(axis=1)/(np.cos(np.radians(xr_da_latcorr[lat_str])).sum()*xr_da_latcorr[lon_str].shape[0])
    if len(xr_da.shape) == 2:
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1).sum(axis=0)/(np.cos(np.radians(xr_da_latcorr[lat_str])).sum()*xr_da_latcorr[lon_str].shape[0])        
    return xr_da_fldmean_latcorr


def clus_to_str(clus):
    """
    Convert array of node numbers to a string
    """
    clus_str = str(clus.values)
    clus_str_corr = (clus_str.replace(".", "").replace("[", "").replace("]", "").replace(" ", "_").replace("__", "_"))
    if clus_str_corr[0] == "_":
        clus_str_corr = clus_str_corr[1:]
    return clus_str_corr

def calc_blocked_days_clus(persis_thresh, SOM_nodes, node_cluster_set_test_str):
    """
    Calculate the number of blocked days in the cluster
    NB this function is fairly slow - perhaps a list comprehension could speed it up?
    SOM_nodes - list of labelled SOM nodes for each day [1,2,9...,7,6]
    blocked_days - arrangement of blocked days in the GTD
    persis_thresh - threshold for number of days to limit blocking
    node_cluster_set_test_str - the set of node clusters being tested, stored as a list of strings ['[1, 2, 5]', '[1, 4]', ... , '[1]']
    """
    #define the new set of blocked days
    blocked_days_clus = np.zeros((SOM_nodes.shape[0]))
    #set blocking for the five day clusters where the given SOM cluster is in the set of node clusters
    for i in range(SOM_nodes.shape[0]-persis_thresh):
            SOM_cluster = str(np.unique(SOM_nodes[i:i+persis_thresh]))
            #print(str(SOM_cluster).replace(' ', ','), str(node_cluster_set_test_str.values).replace('.', '').replace(' ', ','))
            #print(clus_to_str(xr.DataArray(SOM_cluster)), clus_to_str((node_cluster_set_test_str)))
            #print(clus_to_str(xr.DataArray(SOM_cluster)) == clus_to_str((node_cluster_set_test_str)))
            #print(node_cluster_set_test_str)
            if clus_to_str(xr.DataArray(SOM_cluster)) == clus_to_str((node_cluster_set_test_str)):
                blocked_days_clus[i:i+persis_thresh] = 1
            #if SOM_cluster in node_cluster_set_test_str:
            #    blocked_days_clus[i:i+persis_thresh] = 1
    return blocked_days_clus

In [ ]:
files = sorted(glob.glob("/rds/general/user/cmt3718/ephemeral/psl-cmip6-4xco2-day/*LTDManom*"))
zg_dtrnd_fstr = files[1]

if "zg" in zg_dtrnd_fstr:
    var_str = "zg"
if "psl" in zg_dtrnd_fstr:
    var_str = "psl"

In [ ]:

if "variant_label" in ds.attrs:
    ens = ds.attrs['variant_label']
else:
    for i in range(20):
        if f"/r{i}i" in zg_dtrnd_fstr:
            ens = f"r{i}i"+zg_dtrnd_fstr.split(f"r{i}i")[1].split("/")[0]  
        elif f"_r{i}i" in zg_dtrnd_fstr:
            ens = f"r{i}i"+zg_dtrnd_fstr.split(f"r{i}i")[1].split("_")[0]
    
if "source_id" in ds.attrs:
    mdl = ds.attrs['source_id']
elif "model_id" in ds.attrs:
    mdl = ds.attrs['model_id']
elif "source" in ds.attrs:
    mdl = ds.attrs['source']    
years = str(int(ds['time.year'].min()))+"-"+str(int(ds['time.year'].max()))
print(mdl, ens, years)

if "psl" in zg_dtrnd_fstr:
    var_str = "psl"
    codebook_set_str = "/rds/general/user/cmt3718/home/downloads/psl_SOM_5x4_ERA5_JJA_1979-2019_codebook.nc"
    clus_set_str = "/rds/general/user/cmt3718/home/downloads/F1_cv_era5_reanal_5x4_EUR_1979-2019_JJA_extd_msl_tot.nc"
    codebook = xr.open_dataset(codebook_set_str)[f'{var_str}_5x4_codebook']
    codebook_flat = codebook.values.reshape(codebook.shape[0]*codebook.shape[1], codebook.shape[2])    
if "zg" in zg_dtrnd_fstr:
    var_str = "zg"
    codebook_set_str = "/rds/general/user/cmt3718/home/downloads/zg_SOM_5x4_ERA5_JJA_1979-2019_codebook.nc"
    ng_set_str = "/rds/general/user/cmt3718/home/downloads/F1_cv_era5_reanal_5x4_EUR_1979-2019_JJA_extd_z_tot.nc"
    codebook = xr.open_dataset(codebook_set_str)[f'{var_str}_5x4_codebook']/1
    codebook_flat = codebook.values.reshape(codebook.shape[0]*codebook.shape[1], codebook.shape[2]
                                            
                                            
ng_set = xr.open_dataset(ng_set_str)['set'] # set of node groups that defines the blocking index                                      

In [ ]:

yrs_avg = 24 # number of years to average across for detrending geopotential height over the domain - must be an even number
moving_yrs = 1 # number of years to move across by 

persis_thresh = 5 # persistence threshold of five days for a blocking event
rownum, colnum = 5, 4

#zg_dtrnd_fstr is the input file of geopotential height anomalies (can also use MSLP anomalies) with a 2x2 grid resolution

ds = xr.open_dataset(zg_dtrnd_fstr) # dataset saved used to identify model and ensemble
da = sel_JJA_extd(xr.open_dataset(zg_dtrnd_fstr)[var_str].squeeze().sel(lat = np.arange(30, 78, 2), lon = np.arange(-10, 42, 2))) # da used to select dataset, time period and domain


In [ ]:
#detrend if using geopotential height
if var_str == "zg":
    da_dtrnd = detrend(da, yrs_avg, moving_yrs)
    da = da_dtrnd.copy()
da_reshaped = da.values.reshape(da.shape[0], da.shape[1]*da.shape[2])


In [ ]:
#using the reshaped data, identify the BMU for each day in the dataset from the codebook
bmu_arr = Identify_BMU_from_codebook(codebook_flat, da_reshaped, rownum, colnum)
clus_set = xr.open_dataset(clus_set_str)['set']
bmu_da = xr.DataArray(bmu_arr).rename({"dim_0": "time"})    
bmu_da['time'] = da['time'] 
#specify years across dataset
styr, endyr = int(da['time.year'].min()), int(da['time.year'].max())
years = f"{styr}-{endyr}"


bmu_da = bmu_da.assign_attrs({"desc": f"SOM mapping for the 5x4 {var_str} SOM trained on ERA5 JJA 1979-2019 data", "model": mdl, "ensemble": ens, "years": years})
bmu_da.to_netcdf(f"/rds/general/user/cmt3718/home/SOM-BI/Clusters/hst+rcp85/{mdl}/{mdl}_{ens}_{years}_SOM_mapping_5x4_{var_str}_era5.nc")


In [ ]:

#for each node group in the node group set, identify which blocking indices fit with the node group
#the total classification of blocked days is the sum of these.
#Different types of blocking event can be identified through K-means clustering. This calculation and the associated plot for the trends (as per figure 11 in Thomas et al. 2021) can be found in SOM-BI_K-means_codebook_ERA5.ipynb
for ng in ng_set[:]:
    ng_str = ng_to_str(ng)
    print(ng_str)
    blocked_days_clus = calc_blocked_days_clus(persis_thresh, bmu_da, ng)
    da = xr.DataArray(blocked_days_clus, name = clus_str, dims = {"time": bmu_da['time']}) # file with length of the time and 1s for where that node group occurs across the period for each day
    da['time'] = bmu_da['time']
    da.to_netcdf(f"/rds/general/user/cmt3718/home/SOM-BI/Clusters/{sim}/{mdl}/{mdl}_{ens}_{years}_SOM-BI_5x4_{var_str}_era5_{ng_str}.nc")